In [1]:
import copy
import pandas as pd
import json
import sqlite3
from pathlib import Path

In [2]:
# select here if you already put the avatar information in the database
avatars_in_DB = True

In [3]:
print("For which experiment would you like to add Data to the Database?")
experiment = int(input())
if experiment == 1:
    db_path = Path('E:/HumanA/Data/DataBase/HumanA_Exp1.db')
    path = Path('E:/HumanA/Data/Exp1/AllAvatarsList_Coordinates_Exp1.json')
elif experiment == 2:
    db_path = Path('E:/HumanA/Data/DataBase/HumanA_Exp2.db')
    path = Path('E:/HumanA/Data/Exp2/AllAvatarsList_Coordinates_Exp2.json')

For which experiment would you like to add Data to the Database?


In [4]:
def avatarInDB(avatarID):
    # instruction to select all data from database
    sql_instruction = """
    SELECT AvatarID FROM avatars
    """

    # show result
    cr.execute(sql_instruction)
    content = cr.fetchall()
    if (avatarID,) in content:
        return True
    else:
        return False

In [5]:
def _extract_avatars(avatar):
    def get(field:str):
        if '_x' in field:
            return avatar[field[:-2]]['x']
        elif '_y' in field:
            return avatar[field[:-2]]['y']
        elif '_z' in field:
            return avatar[field[:-2]]['z']
        else:
            return avatar[field]
    return (get(field)for field in AVATAR_FIELDS)

In [6]:
if not db_path or not db_path.exists():
    db_path = ':memory:'


if not path.exists():
    raise Exception('Path not exists')  

In [7]:
connection=sqlite3.connect(db_path)
cr=connection.cursor()

In [8]:
AVATAR_FIELDS = (
    'AvatarCenterWorld_x',
    'AvatarCenterWorld_y',
    'AvatarCenterWorld_z',
    'AvatarPositionGlobal_x',
    'AvatarPositionGlobal_y',
    'AvatarPositionGlobal_z',
    'AvatarPositionLocal_x',
    'AvatarPositionLocal_y',
    'AvatarPositionLocal_z',
    'AvatarRotationGlobal_x',
    'AvatarRotationGlobal_y',
    'AvatarRotationGlobal_z',
    'AvatarRotationLocal_x',
    'AvatarRotationLocal_y',
    'AvatarRotationLocal_z',
    'AvatarRotationVector3Global_x',
    'AvatarRotationVector3Global_y',
    'AvatarRotationVector3Global_z',
    'AvatarRotationVector3Local_x',
    'AvatarRotationVector3Local_y',
    'AvatarRotationVector3Local_z',
    'AvatarLossyScale_x',
    'AvatarLossyScale_y',
    'AvatarLossyScale_z',
    'AvatarLocalScale_x',
    'AvatarLocalScale_y',
    'AvatarLocalScale_z',
    'AvatarForwardDirection_x',
    'AvatarForwardDirection_y',
    'AvatarForwardDirection_z',
    'AvatarUpDirection_x',
    'AvatarUpDirection_y',
    'AvatarUpDirection_z',
    'AvatarRightDirection_x',
    'AvatarRightDirection_y',
    'AvatarRightDirection_z',
)

In [9]:
# create instruction for creating table with avatar coordinates
avatar_structure = ',\n'.join(f"{field} NUMERIC" for field in AVATAR_FIELDS)
sql_instruction = f"""
CREATE TABLE IF NOT EXISTS avatars(
AvatarID INTEGER NOT NULL UNIQUE,
AvatarName TEXT,
{avatar_structure},
Category INTEGER,
PRIMARY KEY("AvatarID")
);

"""

cr.execute(sql_instruction)

In [10]:
if __name__ == '__main__':

    if not avatars_in_DB:

        with open(path) as file:
            data = json.load(file)

        values = (
                    str((data[avatar]['AvatarID'], data[avatar]['AvatarName'],) + 
                        tuple(_extract_avatars(avatar=data[avatar])))
                        for avatar in data
                )

        sql_instruction = f"""INSERT INTO avatars
                            (AvatarID, AvatarName, {', '.join(AVATAR_FIELDS)})
                            VALUES {','.join(values)}
                            ;"""

        cr.execute(sql_instruction)

        connection.commit()


        sql_instruction = """
            UPDATE avatars 
            SET Category = 0 
            WHERE AvatarName LIKE '%Sa%'
        """
        cr.execute(sql_instruction)

        sql_instruction = """
            UPDATE avatars 
            SET Category = 1 
            WHERE AvatarName LIKE '%Cma%'
        """
        cr.execute(sql_instruction)
        connection.commit()
        connection.close()